## Регрессия с помощью k-соседей
Нам понадобится решать задачу регрессии с помощью метода k ближайших соседей — воспользуйтесь для этого классом sklearn.neighbors.KNeighborsRegressor. Метрика задается с помощью параметра metric, нас будет интересовать значение ’minkowski’. Параметр метрики Минковского задается с помощью параметра p данного класса.

In [43]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import load_boston
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold #Кросс-валидация
import pandas as pd
import numpy as np

    Приведем признаки в выборке к одному масштабу при помощи функции sklearn.preprocessing.scale.
    
    Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (используйте функцию numpy.linspace). 
    
    Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки scikit-learn версии 0.18.1 и выше необходимо указывать scoring='neg_mean_squared_error'). 
    
    Качество оценивайте, как и в предыдущем задании, с помощью кросс-валидации по 5 блокам с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).

In [64]:
data = load_boston()
#Обработка данных
X = data['data']
y = data['target']
y = scale(y)

#Разбиение на 5 групп
kf = KFold(n_splits = 5, random_state=42, shuffle=True)

#Обучение модели
mass = []
for i in np.linspace(1, 10, 200):
    rg = KNeighborsRegressor(n_neighbors=5, weights='distance', p=i)
    rg.fit(X, y)
    #Проверка качетсва разбиения модели на наборах
    scores = cross_val_score(rg, X, y, cv=kf)
    mass.append(scores)


In [65]:
data = pd.DataFrame(mass, index=range(1,201))
print(data.mean(axis=1))
print('Качество на кросс-валидации оказалось оптимальным при p=1')

1      0.643497
2      0.639041
3      0.636697
4      0.629828
5      0.626985
         ...   
196    0.504215
197    0.504201
198    0.504188
199    0.504036
200    0.504028
Length: 200, dtype: float64
Качество на кросс-валидации оказалось оптимальным при p=1


###### Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского: